In [1]:
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go

# Read Data

- This section takes in the `.csv` files that contain the tree loss from 2001 to 2020 as well as another `.csv` file that contains the country code information.
- We then create the `merged_df` that combines the country codes that were contained in the tree loss data set with the country code information so that it can be properly mapped or named in the visuals.

In [2]:
tree_loss_df = pd.read_csv('./output_data/prediction_ByRegion_exp_smooth.csv')

In [3]:
tree_loss_df.head()

,CountryCode,Year,TreeCoverLoss_ha
0,ABW,2021,0.080184
1,ABW,2022,-0.018594
2,ABW,2023,-0.117372
3,ABW,2024,-0.216149
4,ABW,2025,-0.314927


In [4]:
country_code_info_df = pd.read_csv('input_data/country_code_info.csv')

In [5]:
country_code_info_df.head()

,name,alpha-2,alpha-3,country-code,iso_3166-2,region,sub-region,intermediate-region,region-code,sub-region-code,intermediate-region-code
0,Afghanistan,AF,AFG,4,ISO 3166-2:AF,Asia,Southern Asia,NaN,142.0,34.0,NaN
1,Åland Islands,AX,ALA,248,ISO 3166-2:AX,Europe,Northern Europe,NaN,150.0,154.0,NaN
2,Albania,AL,ALB,8,ISO 3166-2:AL,Europe,Southern Europe,NaN,150.0,39.0,NaN
3,Algeria,DZ,DZA,12,ISO 3166-2:DZ,Africa,Northern Africa,NaN,2.0,15.0,NaN
4,American Samoa,AS,ASM,16,ISO 3166-2:AS,Oceania,Polynesia,NaN,9.0,61.0,NaN


In [6]:
merged_df = tree_loss_df.merge(
    country_code_info_df[['alpha-3', 'region', 'sub-region']],
    left_on='CountryCode',
    right_on='alpha-3',
    how='inner'
)

In [7]:
merged_df.head()

,CountryCode,Year,TreeCoverLoss_ha,alpha-3,region,sub-region
0,ABW,2021,0.080184,ABW,Americas,Latin America and the Caribbean
1,ABW,2022,-0.018594,ABW,Americas,Latin America and the Caribbean
2,ABW,2023,-0.117372,ABW,Americas,Latin America and the Caribbean
3,ABW,2024,-0.216149,ABW,Americas,Latin America and the Caribbean
4,ABW,2025,-0.314927,ABW,Americas,Latin America and the Caribbean


# Visualize tree cover loss by country

In [8]:
fig = px.choropleth(
    merged_df,
    locations="CountryCode",                  
    locationmode="ISO-3",                    
    color="TreeCoverLoss_ha",               
    animation_frame="Year",                
    title="Global Tree Cover Loss by Country(2001-2020)", 
    color_continuous_scale="Reds",     
    labels={"TreeCoverLoss_ha": "Tree Cover Loss (ha)"}, 
    hover_name="region"               
)

fig.show()

In [9]:
top_10_countries_per_year = (
    merged_df.groupby("Year", group_keys=False)
    .apply(lambda x: x.nlargest(10, "TreeCoverLoss_ha"))
    .reset_index()
)

fig = px.bar(
    top_10_countries_per_year,
    x="TreeCoverLoss_ha",           
    y="CountryCode",           
    color="region",             
    animation_frame="Year",       
    title="Top 10 Countries by Tree Cover Loss (2001-2020)",
    labels={"TreeCoverLoss_ha": "Tree Cover Loss (ha)", "CountryCode": "Country Code"},
    orientation="h",           
)

fig.update_layout(
    xaxis_title="Tree Cover Loss (ha)", 
    yaxis_title="Country Code", 
    yaxis=dict(categoryorder="total ascending"),  
    showlegend=True           
)

fig.show()

# Visualize by State

In [10]:
region_year_loss = merged_df.groupby(['Year', 'region'])['TreeCoverLoss_ha'].sum().reset_index()

merged_df_with_year_loss = merged_df.merge(
    region_year_loss,
    on=['Year', 'region'],
    suffixes=('', '_total')
)

final_map_df = merged_df_with_year_loss.drop_duplicates(subset=["CountryCode", "Year"])

fig = px.choropleth(
    final_map_df,
    locations="alpha-3",              
    locationmode="ISO-3",                
    color="TreeCoverLoss_ha_total",    
    hover_name="region",          
    animation_frame="Year",      
    title="Global Tree Cover Loss by Region (2001-2020)", 
    color_continuous_scale="Reds",   
    labels={"TreeCoverLoss_ha_total": "Total Tree Cover Loss (ha)"}
)

fig.show()

In [11]:
fig = px.bar(
    region_year_loss,
    x="TreeCoverLoss_ha",       
    y="region",                
    color="region",            
    animation_frame="Year",      
    title="Global Tree Cover Loss by Region (2001-2020)",  
    labels={"TreeCoverLoss_ha": "Tree Cover Loss (ha)", "region": "Region"},
    orientation="h"           
)

fig.update_layout(
    xaxis_title="Tree Cover Loss (ha)",  
    yaxis_title="Region",         
    yaxis=dict(categoryorder="total ascending"),  
    showlegend=False              
)

fig.show()


# Visualize by sub-region

In [12]:
subregion_year_loss = merged_df.groupby(['Year', 'sub-region'])['TreeCoverLoss_ha'].sum().reset_index()

merged_df_with_subregion_loss = merged_df.merge(
    subregion_year_loss,
    on=['Year', 'sub-region'],
    suffixes=('', '_total') 
)

final_map_df = merged_df_with_subregion_loss.drop_duplicates(subset=["CountryCode", "Year"])

fig = px.choropleth(
    final_map_df,
    locations="alpha-3",                  
    locationmode="ISO-3",              
    color="TreeCoverLoss_ha_total",       
    hover_name="sub-region",             
    animation_frame="Year",             
    title="Global Tree Cover Loss by Sub-Region (2001-2020)", 
    color_continuous_scale="Reds",     
    labels={"TreeCoverLoss_ha_total": "Total Tree Cover Loss (ha)"}
)

fig.show()


In [13]:
fig = px.bar(
    subregion_year_loss,
    x="TreeCoverLoss_ha",        
    y="sub-region",             
    color="sub-region",         
    animation_frame="Year",    
    title="Global Tree Cover Loss by Sub-Region (2001-2020)", 
    labels={"TreeCoverLoss_ha": "Tree Cover Loss (ha)", "sub-region": "Sub-Region"},
    orientation="h"         
)

fig.update_layout(
    xaxis_title="Tree Cover Loss (ha)", 
    yaxis_title="Sub-Region",  
    yaxis=dict(categoryorder="total ascending"), 
    showlegend=False   
)

fig.show()

# Visualize Individual Countries

In [14]:
def create_scatter_plot(selected_country):
    country_data = merged_df[merged_df['CountryCode'] == selected_country]
    
    fig = go.Figure(
        data=go.Scatter(
            x=country_data['Year'],
            y=country_data['TreeCoverLoss_ha'],
            mode='lines+markers',
            marker=dict(color='blue'),
            name=selected_country
        )
    )
    
    fig.update_layout(
        title=f"Tree Cover Loss for {selected_country} by Year",
        xaxis_title="Year",
        yaxis_title="Tree Cover Loss (ha)",
        showlegend=False
    )
    
    return fig


default_country = merged_df['CountryCode'].iloc[0]
fig = create_scatter_plot(default_country)


dropdown_buttons = [
    {
        'label': country,
        'method': 'update',
        'args': [
            {'x': [merged_df[merged_df['CountryCode'] == country]['Year']],
             'y': [merged_df[merged_df['CountryCode'] == country]['TreeCoverLoss_ha']]},  
            {'title': f"Tree Cover Loss for {country} by Year", 'showlegend': False} ]
    }
    for country in merged_df['CountryCode'].unique()
]


fig.update_layout(
    updatemenus=[{
        'buttons': dropdown_buttons,
        'direction': 'down',
        'showactive': True,
        'x': 0.17,
        'xanchor': 'left',
        'y': 1.15,
        'yanchor': 'top'
    }]
)

# Show the plot
fig.show()


# Sum of all Tree Loss Worldwide

In [15]:
total_tree_loss_per_year = merged_df.groupby('Year')['TreeCoverLoss_ha'].sum().reset_index()


fig = go.Figure(
    data=go.Scatter(
        x=total_tree_loss_per_year['Year'], 
        y=total_tree_loss_per_year['TreeCoverLoss_ha'],
        mode='lines+markers',  
        marker=dict(color='blue'),
        name='Total Tree Cover Loss'
    )
)


fig.update_layout(
    title="Total Tree Cover Loss Worldwide by Year",
    xaxis_title="Year",
    yaxis_title="Total Tree Cover Loss (ha)",
    showlegend=False
)

fig.show()
